In [7]:
from Code.lib.plot_utils import PlotUtility
from Code.lib.time_utils import TimeUtility
from Code.lib.retrieve_data import DataRetrieve, ComputeTarget
from Code.lib.candle_indicators import CandleIndicators
from Code.lib.transformers import Transformers
from Code.lib.ta_momentum_studies import TALibMomentumStudies
from Code.lib.model_utils import ModelUtility, TimeSeriesSplitImproved
from Code.lib.feature_generator import FeatureGenerator
from Code.utilities.stat_tests import stationarity_tests
from Code.lib.config import current_feature, feature_dict
from Code.models import models_utils
from Code.lib.model_algos import AlgoUtility

plotIt = PlotUtility()
timeUtil = TimeUtility()
ct = ComputeTarget()
candle_ind = CandleIndicators()
dSet = DataRetrieve()
taLibMomSt = TALibMomentumStudies()
transf = Transformers()
modelUtil = ModelUtility()
featureGen = FeatureGenerator()
dSet = DataRetrieve()
modelAlgo = AlgoUtility()

In [8]:
issue = "TLT"

In [20]:
df = dSet.read_issue_data(issue)

Successfully retrieved data series for TLT


In [29]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg




In [28]:
raw = DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
values = raw.values
data = series_to_supervised(values, 2, 2)
print(raw)
print(data)

   ob1  ob2
0    0   50
1    1   51
2    2   52
3    3   53
4    4   54
5    5   55
6    6   56
7    7   57
8    8   58
9    9   59
   var1(t-2)  var2(t-2)  var1(t-1)  var2(t-1)  var1(t)  var2(t)  var1(t+1)  \
2        0.0       50.0        1.0       51.0        2       52        3.0   
3        1.0       51.0        2.0       52.0        3       53        4.0   
4        2.0       52.0        3.0       53.0        4       54        5.0   
5        3.0       53.0        4.0       54.0        5       55        6.0   
6        4.0       54.0        5.0       55.0        6       56        7.0   
7        5.0       55.0        6.0       56.0        7       57        8.0   
8        6.0       56.0        7.0       57.0        8       58        9.0   

   var2(t+1)  
2       53.0  
3       54.0  
4       55.0  
5       56.0  
6       57.0  
7       58.0  
8       59.0  


In [30]:
dataSet = dSet.set_date_range(df, '2016-02-14','2019-02-14')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4168 entries, 2002-07-26 to 2019-02-14
Data columns (total 6 columns):
Open        4168 non-null float64
High        4168 non-null float64
Low         4168 non-null float64
Close       4168 non-null float64
AdjClose    4168 non-null float64
Volume      4168 non-null float64
dtypes: float64(6)
memory usage: 387.9 KB


In [35]:
dff = df['Close']
values = dff.values
values

array([  82.51  ,   81.42  ,   81.52  , ...,  121.555 ,  121.1   ,
        121.7633])

In [33]:
data = series_to_supervised(values, 2)


IndexError: tuple index out of range

In [24]:
df.shape[1]

7

In [ ]:
print(data)